# AfriSAR Search and Visualize

Authors: Nikita Susan (UAH), Aimee Barciauskas (DevSeed)

Date: January 17, 2023

Description:

## Run This Notebook
To access and run this tutorial within MAAP's Algorithm Development Environment (ADE), please refer to the ["Getting started with the MAAP"](https://docs.maap-project.org/en/latest/getting_started/getting_started.html) section of our documentation.

Disclaimer: it is highly recommended to run a tutorial within MAAP's ADE, which already includes packages specific to MAAP, such as maap-py. Running the tutorial outside of the MAAP ADE may lead to errors.

## About the Data

This dataset provides gridded estimates of aboveground biomass (AGB) for four sites in Gabon at 0.25 ha (50 m) resolution derived with field measurements and airborne LiDAR data collected from 2010 to 2016. The sites represent a mix of forested, savannah, and some agricultural and disturbed landcover types: Lope site, within Lope National Park; Mabounie, mostly forested site; Mondah Forest, protected area; and the Rabi forest site, part of the Smithsonian Institution of Global Earth Observatories world-wide network of forest plots. Plot-level biophysical measurements of tree diameter and tree height (or estimated by allometry) were performed at 1 ha and 0.25 ha scales on multiple plots at each site and used to derive AGB for each tree and then summed for each plot. Aerial LiDAR scans were used to construct digital elevation models (DEM) and digital surface models (DSM), and then the DEM and DSM were used to construct a canopy height model (CHM) at 1 m resolution. After checking site-plot locations against the CHM, mean canopy height (MCH) was computed over each 0.25 ha. A single regression model relating MCH and AGB estimates, incorporating local height based on the trunk DBH (HD) relationships, was produced for all sites and combined with the CHM layer to construct biomass maps at 0.25 ha resolution. (Source: [AfriSAR AGB User Guide](https://daac.ornl.gov/AFRISAR/guides/AfriSAR_AGB_Maps.html))

## Additional Resources
- [AfriSAR AGBD Dataset Landing Page](https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1681)
- [Earthdata Search](https://search.earthdata.nasa.gov/search?q=AfriSAR_AGB_Maps_1681)

## Import and Install Packages

In [ ]:
# Uncomment and run the line below if missing packages
!pip install pystac-client rioxarray hvplot

In [2]:
import rioxarray
import rasterio as rio
from pystac_client import Client
import hvplot.xarray

import warnings
warnings.filterwarnings("ignore")

## Declare your collection of interest

You can discover available collections the follwoing ways:

1. Use the {STAC_API_URL}/collections API endpoint (JSON response)
2. Programmatically using pystac (see example in the list-collections.ipynb notebook)
3. In the STAC Browser : http://delta-staging-stac-browser.s3-website-us-east-1.amazonaws.com/


In [3]:
STAC_API_URL = 'https://stac.maap-project.org/'
collection = 'AfriSAR_AGB_Maps_1681'

In [4]:
# custom headers
headers = []

cat = Client.open(STAC_API_URL, headers=headers)


## Listing all the available collections

In [5]:
for collection in cat.get_collections():
    print(collection)


<CollectionClient id=Landsat8_SurfaceReflectance>
<CollectionClient id=Global_PALSAR2_PALSAR_FNF>
<CollectionClient id=Global_Forest_Change_2000-2017>
<CollectionClient id=AFRISAR_DLR2>
<CollectionClient id=AfriSAR_UAVSAR_KZ>
<CollectionClient id=AfriSAR_UAVSAR_Ungeocoded_Covariance>
<CollectionClient id=AfriSAR_UAVSAR_Normalization_Area>
<CollectionClient id=AfriSAR_UAVSAR_Geocoded_SLC>
<CollectionClient id=AfriSAR_UAVSAR_Geocoded_Covariance>
<CollectionClient id=GlobCover_09>
<CollectionClient id=GlobCover_05_06>
<CollectionClient id=GEDI_CalVal_Field_Data>
<CollectionClient id=AfriSAR_UAVSAR_Coreg_SLC>
<CollectionClient id=GEDI_CalVal_Lidar_Data_Compressed>
<CollectionClient id=GEDI_CalVal_Lidar_Data>
<CollectionClient id=ABoVE_UAVSAR_PALSAR>
<CollectionClient id=AFRISAR_DLR>
<CollectionClient id=BIOSAR1>
<CollectionClient id=icesat2-boreal>
<CollectionClient id=ICESat2_Boreal_AGB_tindex_average>
<CollectionClient id=NCEO_Africa_AGB_100m_2017>
<CollectionClient id=Paraguay_Country_P

## Getting the collection of intrest 'AfriSAR_AGB_Maps_1681'

In [6]:
collection = cat.get_collection('AfriSAR_AGB_Maps_1681')
collection

APIError: {"code":"NotFoundError","description":"Collection AfriSAR_AGB_Maps_1681 does not exist."}

## Use satsearch to discover items in the 'AfriSAR_AGB_Maps_1681' collection

In [ ]:
search = cat.search(
    max_items = 15,
    limit = 5,
    collections = collection,
)

## Adding items to an iterable

In [ ]:
items = [i.to_dict() for i in search.get_all_items()]
items[0]

## Extracting s3 link and reading it with rioxarray

In [ ]:
item = items[0]
s3_link = item['assets']['data']['href']
da = rioxarray.open_rasterio(s3_link)
da = da.squeeze('band', drop=True)
da

## Visualizing the read data with hvplot

In [ ]:
ds_masked = da.where(da != da._FillValue)

ds_masked.hvplot(
    'x', 'y', 
    cmap='viridis',
    frame_height=400,
    frame_width=400
).redim.range(value=(0,da.max().values))